# Input MQTT

Reads from MQTT topics in batches or streams

In [ ]:
!pip install paho-mqtt flask

In [ ]:
import logging
import os
import re
import sys
import random
from paho.mqtt import client as mqtt_client
import threading
from flask import Flask
import queue

In [ ]:
# mqtt port
port = int(os.environ.get('port',1883))

# mqtt server
broker = os.environ.get('broker')

# mqtt topic
topic = os.environ.get('topic')

# mqtt user name
username = os.environ.get('username', '')

# mqtt password
password = os.environ.get('password', '')

In [ ]:
parameters = list(
    map(lambda s: re.sub('$', '"', s),
        map(
            lambda s: s.replace('=', '="'),
            filter(
                lambda s: s.find('=') > -1 and bool(re.match(r'[A-Za-z0-9_]*=[.\/A-Za-z0-9]*', s)),
                sys.argv
            )
    )))

port = int(port)

for parameter in parameters:
    logging.warning('Parameter: ' + parameter)
    exec(parameter)

In [ ]:
def connect_mqtt():
    def on_connect(client, userdata, flags, rc):
        if rc == 0:
            print("Connected to MQTT Broker!")
        else:
            print("Failed to connect, return code %d\n", rc)
    # Set Connecting Client ID
    client = mqtt_client.Client(f'python-mqtt-{random.randint(0, 1000)}')
    client.username_pw_set(username, password)
    client.on_connect = on_connect
    client.connect(broker, port)
    return client

In [ ]:
client = connect_mqtt()
q = queue.Queue()

In [ ]:
def subscribe(client: mqtt_client):
    def on_message(client, userdata, msg):
        q.put(msg.payload.decode())
    client.subscribe(topic)
    client.on_message = on_message

subscribe(client)

x = threading.Thread(target=client.loop_forever)
x.start()


In [ ]:
app = Flask(__name__)

@app.route('/')
def index():
    return q.get()

app.run(host='0.0.0.0', port=8080)